## For Lesson 1

In [ ]:
x = RealField(3.32*227).pi().exp()
s = f"e^pi = {x}"
# print(s)

In [ ]:
len(s) - 7

## For lesson 2

In [ ]:
def zeta_v1(s, eps=1e-5):
    r"""
    Compute the Riemann zeta function.
    
    INPUT:
    
    - ``s`` -- complex number, argument
    - ``eps`` -- real number, desired error estimate
    
    OUTPUT: complex number of same type as input
    """
    if not isinstance(s,sage.rings.complex_number.ComplexNumber):
        raise ValueError("Need a complex number!")
    if s.real() <= 1:
        raise ValueError("Complex number need ot have real part greater than 1")
    # Crude error bound, simply estimating the last term
    nmax = ceil(eps**(-1/s.real()))
    return sum([s.parent()(n)**(-s) for n in range(1, nmax)])
    
    

In [ ]:
zeta_v1(CC(3), eps=1e-10) - CC(zeta(3))

In [ ]:
zeta_v1(CC(2,1))

In [ ]:
plot([lambda x: zeta_v1(CC(x, 1)).real(), lambda x: zeta_v1(CC(2,x)).real()], 2, 10, legend_label=['$\\zeta(x+i)$', r'$\zeta(2+xi)$'])

## Lesson 3

In [ ]:
var('s, t, u')
f = (s + t*u) / sqrt(s^2 + t^2 + u^2); f
diff(f, u).substitute(s=0, u=1, t=1) 

so correct answer is $\frac{1}{2\sqrt{2}}$

In [ ]:
var('x, t')
F = t * e**(x*t) / (e**t - 1); F

In [ ]:
F.taylor(t, 0, 1).coefficient(t)

In [ ]:
def Bernoulli_pol(k):
    var('x, t')
    F = t * e**(x*t) / (e**t - 1)
    return factorial(k)*F.taylor(t, 0, k).coefficient(t^k)

In [ ]:
var('y')
bernoulli_polynomial(y, 1)

In [ ]:
[Bernoulli_pol(k) - bernoulli_polynomial(x, k) for k in range(2, 50, 2)]

In [ ]:
%timeit Bernoulli_pol(10)

In [ ]:
%timeit bernoulli_polynomial(x, 10)

# Examples for Lesson 4

In [ ]:
def pochhammer(s, j):
    return gamma(s + j) / gamma(s)

def zeta_euler_mclaurin(s, k, a=10):
    F = s.parent()
    ca = F(a)
    term0 = sum(F(n)**-s for n in range(1, a))
    term1 = 1 / (s - 1) * F(a)**(1 - s)
    term2 = 1/2 * F(a)**-s
    term3 = sum((-1)**(j - 1) * pochhammer(s, j - 1) * bernoulli(j) / factorial(j)
                 *ca**(-s - j + 1) for j in range(2, k + 1))
    # print(term1, term2, term3)
    return term0 + term1 + term2 - term3

In [ ]:
CF = ComplexField(53)
diffs = [lambda t: abs(zeta_euler_mclaurin(CF(1, t), k, a=3) - 
                       CF(zeta(CF(1,t)))) for k in [10, 15]]
diffs = [lambda t: abs(zeta_euler_mclaurin(CF(1, t), 10, a=3) - 
                       CF(zeta(CF(1, t)))),
         lambda u: abs(zeta_euler_mclaurin(CF(1, u), 15, a=3) - 
                       CF(zeta(CF(1, u))))]

In [ ]:
plot([diffs[0], diffs[1]], 1, 5, legend_label=['10', '15'])
# shows that the choice of a is not that important

In [ ]:
def bernoulli_recursive(k):
    if k == 0:
        return 1
    return -sum([binomial(k, j) * bernoulli_recursive(j) / (k - j + 1) for j in range(0, k)])

In [ ]:
[bernoulli_recursive(k) for k in range(10)]

# Examples for Lesson 5

In [ ]:
class ZetaNumerical(SageObject):
    r"""
    Class to compute the Riemann zeta function.
    """
    def __init__(self, prec=53, maxn=100):  # 'dunder' init function
        r"""
        Initialize.
        """
        # define some "private" properties
        self._prec = prec
        self._base_ring = ComplexField(prec)
        self._maxn = maxn

    # convencience methods for accessing (not modifying) private variables
    def prec(self):
        r"""
        Return precision of self.
        """
        return self._prec
    
    def base_ring(self):
        r"""
        Return the base ring of self.
        """
        return self._base_ring
    
    
    def __call__(self,s):  # 'dunder' call function, used when class element is "called"
        # The 'r' in front of """ means that this is a "raw" string,
        # in particular the backslash in '\z' will work here.
        r"""
        Evaluate `\zeta(s)`

        INPUT:

        - ``s`` -- complex number

        EXAMPLE:
        
        sage: z = ZetaNumerical(103)
        sage: z(10)
        ...
        sage: z(x)
        ...
        """
        try: 
            scplx = self._base_ring(s)
        except TypeError:  # Type error is raised when an internal function can not apply on this type
            raise ValueError(f"Could not coerce {z} to a complex number!")
        if scplx.real() > 1:
            return self._sum(scplx)
        return self._euler_mclaurin(scplx)
    
    # Private method starting with '_'. 
    # Should not be called explicitly from outside this class
    def _sum(self, s):
        r"""
        Evaluate self using a "naive" sum.
        """
        return sum([self._base_ring(n)**-s for n in range(1, self._maxn)])
    
    def _euler_mclaurin(self, s):
        r"""
        Evaluate self using the Euler McLaurin summation formula.
        """
        # When writing a class it is useful to populate it with methods you would like
        # and then simply mark these as 
        # raise NotImplementedError("This method has not been implemented yet!")
        return zeta_euler_mclaurin(s, k=25, a=10)
    
    def __repr__(self):
        """
        String representation of self.
        """
        return f"NumericalZ with precision {self._prec}"
    
    def plot_along_vertical(self,x,ylim,num_points=100):
        """
        Plot zeta along on a vertical line.

        INPUT:

        - ``x`` -- real number, the x-coordinate of the line

        - ``ylim`` -- tuple (a,b) with a < b, endpoint of the vertical line

        - ``num_points`` -- integer (default=100), number of points to use
        """
        try: 
            a, b = ylim
        except TypeError:
            raise ValueError("Limits must be  tuple!")
        if a >= b:
            raise ValueError("Endpoints (a, b) must satisfy a < b")
        return parametric_plot((lambda t: self(CC(x, t)).real(),
                                lambda t: Z(CC(x, t)).imag()), (t, a, b))

    def __eq__(self, other):
        if not isinstance(other, ZetaNumerical):
            return False
        return self._prec == other._prec and self._maxn == other._maxn

In [ ]:
Z = ZetaNumerical();Z

In [ ]:
str(Z)

In [ ]:
Z.plot_along_vertical(2, (2, 3))

# Examples for Lesson 6

In [ ]:
# Exclude the point s = 1 from the plot
plot3d(lambda x, y: abs(zeta(CF(x, y))) if (x, y) != (1, 0) else 0, (0, 2), (0, 20), viewer='canvas3d')

In [ ]:
complex_plot(lambda z: zeta(z), (0, 1), (14, 14.5))

In [ ]:
p3 = density_plot(lambda x, y: abs(zeta(CC(x, y))), (0, 1), (10, 15), cmap='jet')
p3

# Exampes for Lesson 7

In [ ]:
%timeit zeta(CC(0.5, 10))

In [ ]:
%timeit zeta_euler_mclaurin(CC(0.5, 10), 10, 2)

In [ ]:
def primes_mod_n(n, m):
    results = {i: 0 for i in range(n)}  # initialize dict
    for p in primes(m):
        results[p % n] += 1
    return results

In [ ]:
primes_mod_n(4, 10000)

In [ ]:
# Combining the plots like this doesn't work since the variable name 
# x is the same. It will only return the last plot...
plot([lambda x: primes_mod_n(4, floor(x))[i] for i in [1, 3]], 1, 100)

In [ ]:
plot([lambda x: primes_mod_n(4, floor(x))[1], lambda y: primes_mod_n(4, floor(y))[3]], 1, 100)
# 
# Trying something like this maybe??
# var('x1, x3')
# plot([lambda eval(f'x{i}'): primes_mod_n(4, floor(eval(f'x{i}')))[i] for i in [1, 3]], 1, 100)

In [ ]:
def reduce_matrix_modn_inZZ(m):
    if m.parent().degree() != 2:
        raise ValueError("This only works for 2 x 2 matrices!")
    n = m.parent().base_ring().characteristic()
    [a, b], [c, d] = [[x % n for x in y] for y in m.list()]
    result =  matrix(ZZ, 2, 2, [[a, b], [c, d]])
    result.set_immutable()
    return result